In [ ]:
import pandas as pd
import cv2
import wget
import shutil
import os
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import numpy as np
import torch
from PIL import Image
# from google.colab import drive
import matplotlib.pyplot as plt

In [10]:
def get_closest_idx(df_ids, row_id, coco_val, train):
    coords = coco_val[row_id][3].split(',')
    w, h = int(coords[2]), coords[3]
    vals = train.iloc[df_ids]
    min_diff = 1e5
    best_id = -1
    for i, val in vals.iterrows():
        diff = abs(abs(val['right'] - val['left']) - w)
        try:
            if diff < min_diff:
                min_diff = diff
                best_id = i
        except:
            print(best_id, df_ids[i])

    return best_id



In [8]:
def get_iou(bb1, bb2):
    # Taken from https://stackoverflow.com/a/42874377
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    # assert bb1['x1'] < bb1['x2']
    # assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

## Train val predictions

In [2]:
res = pd.read_json('/media/hdd/anoskov/ofa_test/OFA/results/refcoco/refcoco_val_predict.json')
res.head()

,uniq_id,box
0,88888880,"[206.33665466308594, 212.4038848876953, 30.758..."
1,88888890,"[119.8824234008789, 77.5023422241211, 49.20937..."
2,88888900,"[468.42266845703125, 342.7523498535156, 23.068..."
3,88888910,"[286.2228698730469, 263.3675842285156, 23.8164..."
4,88888920,"[255.1974334716797, 154.2718963623047, 90.8329..."


In [3]:
res['box'][0]

[206.33665466308594, 212.4038848876953, 30.75826072692871, 15.812764167785645]

In [4]:
train = pd.read_csv('/media/hdd/anoskov/ofa_test/train.csv')
print(train.head())
print(train.shape)

                                               image  width  height  left  \
0  https://toloka-cdn.azureedge.net/wsdmcup2023/0...    640     427   129   
1  https://toloka-cdn.azureedge.net/wsdmcup2023/0...    640     427   424   
2  https://toloka-cdn.azureedge.net/wsdmcup2023/0...    640     428   242   
3  https://toloka-cdn.azureedge.net/wsdmcup2023/0...    640     480    52   
4  https://toloka-cdn.azureedge.net/wsdmcup2023/0...    565     640   316   

   top  right  bottom                                  question  
0  192    155     212               What does it use to breath?  
1   28    427      32                  What can fly in the sky?  
2  149    406     351                Where can i put my flowers  
3  372    636     443  What equipment is used for snowboarding?  
4  323    452     419                  Where can I put my feet?  
(38990, 8)


,image,width,height,left,top,right,bottom,question
20271,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,427,193,204,223,220,What we wear on our heads?
26804,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,539,286,64,387,137,What we wear on our heads?


In [5]:
coco_val = []
with open('/media/hdd/anoskov/ofa_test/OFA/coco_val.tsv') as f:
    while True:
        line = f.readline()
        if not line:
            break
        coco_val.append(line.split('\t')[:4])
print(len(coco_val))

7798


In [6]:
coco_val[0]

['8888888_0', '8888888', 'What we wear on our heads?', '208.0,212.0,30,16']

In [7]:
train[train['question'] == 'What we wear on our heads?'].iloc[0]

image       https://toloka-cdn.azureedge.net/wsdmcup2023/0...
width                                                     640
height                                                    427
left                                                      193
top                                                       204
right                                                     223
bottom                                                    220
question                           What we wear on our heads?
Name: 20271, dtype: object

In [8]:
aaa = train[train['question'] == 'What we wear on our heads?'].index
train.iloc[aaa]

,image,width,height,left,top,right,bottom,question
20271,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,427,193,204,223,220,What we wear on our heads?
26804,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,539,286,64,387,137,What we wear on our heads?


In [9]:
coco_val[0][3].split(',')

['208.0', '212.0', '30', '16']

In [22]:
# used = [0 for i in range(len(coco_val))]

id_pairs = [-1 for i in coco_val]

for row in range(len(coco_val)):
    ques = coco_val[row][2]
    ids = train[train['question'] == ques].index
    df_idx = get_closest_idx(ids, row)
    id_pairs[row] = df_idx



In [36]:
predictions = pd.read_json('/media/hdd/anoskov/ofa_test/OFA/results/refcoco/refcoco_val_predict.json')
predictions.head()

,uniq_id,box
0,88888880,"[206.33665466308594, 212.4038848876953, 30.758..."
1,88888890,"[119.8824234008789, 77.5023422241211, 49.20937..."
2,88888900,"[468.42266845703125, 342.7523498535156, 23.068..."
3,88888910,"[286.2228698730469, 263.3675842285156, 23.8164..."
4,88888920,"[255.1974334716797, 154.2718963623047, 90.8329..."


In [43]:
# coco_val, train
ious = [0 for i in id_pairs]
for i in range(len(id_pairs)):
    row = train.iloc[id_pairs[i]]
    # print(coco_val[i])
    # break
    bb1 = {'x1': row['left'], 'x2': row['right'], 'y1': row['top'], 'y2': row['bottom']}
    pred_row = predictions.iloc[i]['box']
    left = pred_row[0] - pred_row[2] / 2.
    right = pred_row[0] + pred_row[2] / 2.
    top = pred_row[1] - pred_row[3] / 2.
    bot = pred_row[1] + pred_row[3] / 2.

    # print(pred_row)
    # break
    bb2 = {'x1': left, 'x2': right, 'y1': top, 'y2': bot}
    ious[i] = get_iou(bb1, bb2)

In [44]:
sum(ious) / len(ious)

0.6041723592184729

In [19]:
for idx, row in train.iterrows():
    if row['question'] == 'What we wear on our heads?':
        print(idx)

20271
26804


In [25]:
train.iloc[[20271, 26804]]

,image,width,height,left,top,right,bottom,question
20271,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,427,193,204,223,220,What we wear on our heads?
26804,https://toloka-cdn.azureedge.net/wsdmcup2023/0...,640,539,286,64,387,137,What we wear on our heads?


## Test predictions

In [ ]:
%cd OFA/

In [ ]:
import pandas as pd
import cv2
import wget
import shutil
import os
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import numpy as np
import torch
from PIL import Image
# from google.colab import drive
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from transformers import OFATokenizer, OFAModel
from transformers.models.ofa.generate import sequence_generator
# from google.colab.patches import cv2_imshow

from tqdm import tqdm
from fairseq import utils, tasks
from fairseq import checkpoint_utils
from utils.eval_utils import eval_step
from tasks.mm_tasks.refcoco import RefcocoTask
from models.ofa import OFAModel
import base64
from sklearn.model_selection import train_test_split

# Register refcoco task
tasks.register_task('refcoco', RefcocoTask)

# turn on cuda if GPU is available
use_cuda = torch.cuda.is_available()
# use fp16 only when GPU is available
use_fp16 = False

test_public = pd.read_csv('/test_public.csv')

print(test_public.head())
print(test_public.shape)

# if 'imgs' not in os.listdir():
#   os.mkdir('imgs')

# 8909 15809 17110 18019 26028 36668

# not_written_imgs = []

# with open('NOT_WRITTEN.txt') as f:
#     not_written_imgs = f.readlines()

# url_path = 'https://toloka-cdn.azureedge.net/wsdmcup2023/'

# img_paths = Parallel(
#     n_jobs=100)(delayed(wget.download)(url_path + img_url, out='imgs') for img_url in tqdm(not_written_imgs)
# )

# train, val = train_test_split(test_public, test_size=0.2)

# print(f"Splitting into {len(train)} train and {len(val)} val instances")



In [ ]:
img_paths = Parallel(
    n_jobs=100)(delayed(wget.download)(img_url, out='imgs_test_public/') for img_url in tqdm(test_public['image'])
)

In [3]:
len(os.listdir('imgs_test_public/'))

1705

In [ ]:
# not_written_val = []
# not_written_train = []

unique_cnt = 7777777
img_id = 7777777   # find max, probably

f = open("coco_test_public.tsv", "a")

for id, row in tqdm(test_public.iterrows()):
    question = row['question']
    width = abs(row['left'] - row['right'])
    height = abs(row['bottom'] - row['top'])
    x_cent = row['left'] + width / 2.
    y_cent = row['top'] + height / 2.

    img_path = 'OFA/imgs_test_public/' + row['image'][-16:]
    try:
        with open(img_path, "rb") as img_file:
            base_64_str = base64.b64encode(img_file.read())
    except:
        not_written_train.append(img_path)

    # print(str(base_64_str)[2:-1])
    new_line = f"{unique_cnt}_0	{img_id}	{question}	{x_cent},{y_cent},{width},{height}	{str(base_64_str)[2:-1]}\n"
    f.write(new_line)
    unique_cnt += 1
    img_id += 1
    # break


In [5]:
coco_test_public = []
with open('OFA/coco_test_public.tsv') as f:
    while True:
        line = f.readline()
        if not line:
            break
        coco_test_public.append(line.split('\t')[:4])
print(len(coco_test_public))

1705


In [11]:
# used = [0 for i in range(len(coco_val))]

id_pairs = [-1 for i in coco_test_public]

for row in range(len(coco_test_public)):
    ques = coco_test_public[row][2]
    ids = test_public[test_public['question'] == ques].index
    df_idx = get_closest_idx(ids, row, coco_test_public, test_public)
    id_pairs[row] = df_idx



In [13]:
len(id_pairs), id_pairs[3]

(1705, 3)

In [17]:
predictions = pd.read_json('OFA/results/refcoco/test/refcoco_test_public_predict.json')
predictions.head()

,uniq_id,box
0,77777770,"[443.7644958496094, 222.2383270263672, 97.0578..."
1,77777780,"[398.89617919921875, 465.8594665527344, 93.235..."
2,77777790,"[415.24749755859375, 155.4166717529297, 177.10..."
3,77777800,"[331.2920837402344, 184.60108947753906, 78.177..."
4,77777810,"[44.214996337890625, 75.10822296142578, 83.944..."


In [19]:
# coco_val, train
ious = [0 for i in id_pairs]
for i in range(len(id_pairs)):
    row = test_public.iloc[id_pairs[i]]
    # print(coco_val[i])
    # break
    bb1 = {'x1': row['left'], 'x2': row['right'], 'y1': row['top'], 'y2': row['bottom']}
    pred_row = predictions.iloc[i]['box']
    left = pred_row[0] - pred_row[2] / 2.
    right = pred_row[0] + pred_row[2] / 2.
    top = pred_row[1] - pred_row[3] / 2.
    bot = pred_row[1] + pred_row[3] / 2.

    # print(pred_row)
    # break
    bb2 = {'x1': left, 'x2': right, 'y1': top, 'y2': bot}
    ious[i] = get_iou(bb1, bb2)

In [20]:
sum(ious) / len(ious)

0.6016819608221556

In [22]:
ious = np.array(ious)
print(np.mean(ious), np.median(ious) )

0.6016819608221565 0.7405969926409991


In [44]:
for i in range(10, 101, 10):
    print(f'Pecrentle {i}-th : {np.percentile(ious, i)}')

Pecrentle 10-th : 0.0
Pecrentle 20-th : 0.2761937205854278
Pecrentle 30-th : 0.5757064456716211
Pecrentle 40-th : 0.6827042997237863
Pecrentle 50-th : 0.7405969926409991
Pecrentle 60-th : 0.7808215946088883
Pecrentle 70-th : 0.8170010037773711
Pecrentle 80-th : 0.8493271994736944
Pecrentle 90-th : 0.8783222704074543
Pecrentle 100-th : 0.9672587214127595


## Test private

In [45]:
test_private = pd.read_csv('/test_private.csv')
print(test_private.head())
print(test_private.shape)

                                               image  width  height  left  \
0  https://toloka-cdn.azureedge.net/wsdmcup2023/0...    640     536   516   
1  https://toloka-cdn.azureedge.net/wsdmcup2023/0...    640     427     3   
2  https://toloka-cdn.azureedge.net/wsdmcup2023/0...    427     640   253   
3  https://toloka-cdn.azureedge.net/wsdmcup2023/0...    500     375   130   
4  https://toloka-cdn.azureedge.net/wsdmcup2023/0...    640     640   243   

   top  right  bottom                                           question  
0  270    552     309  what do military men use to identify the type ...  
1  214     13     229                       Where do we throw the trash?  
2  446    334     484                     What do you wear on your feet?  
3  268    366     317                    What do we use to slide on snow  
4  244    412     639             What receives our payment for parking?  
(4504, 8)


In [46]:
img_paths = Parallel(
    n_jobs=100)(delayed(wget.download)(img_url, out='imgs_test_private/') for img_url in tqdm(test_private['image'])
)

100%|██████████| 4504/4504 [01:28<00:00, 50.90it/s] 


In [47]:
len(os.listdir('imgs_test_private/'))

4504

In [48]:
# not_written_val = []
# not_written_train = []

unique_cnt = 5555555
img_id = 5555555   # find max, probably

f = open("coco_test_private.tsv", "a")

for id, row in tqdm(test_private.iterrows()):
    question = row['question']
    width = abs(row['left'] - row['right'])
    height = abs(row['bottom'] - row['top'])
    x_cent = row['left'] + width / 2.
    y_cent = row['top'] + height / 2.

    img_path = 'OFA/imgs_test_private/' + row['image'][-16:]
    try:
        with open(img_path, "rb") as img_file:
            base_64_str = base64.b64encode(img_file.read())
    except:
        not_written_train.append(img_path)

    # print(str(base_64_str)[2:-1])
    new_line = f"{unique_cnt}_0	{img_id}	{question}	{x_cent},{y_cent},{width},{height}	{str(base_64_str)[2:-1]}\n"
    f.write(new_line)
    unique_cnt += 1
    img_id += 1
    # break


4504it [00:05, 792.39it/s]


In [49]:
coco_test_private = []
with open('OFA/coco_test_private.tsv') as f:
    while True:
        line = f.readline()
        if not line:
            break
        coco_test_private.append(line.split('\t')[:4])
print(len(coco_test_private))

4504


In [50]:
# used = [0 for i in range(len(coco_val))]

id_pairs = [-1 for i in coco_test_private]

for row in range(len(coco_test_private)):
    ques = coco_test_private[row][2]
    ids = test_private[test_private['question'] == ques].index
    df_idx = get_closest_idx(ids, row, coco_test_private, test_private)
    id_pairs[row] = df_idx

In [51]:
len(id_pairs), id_pairs[3]

(4504, 3)

In [54]:
predictions = pd.read_json('OFA/results/refcoco/refcoco_test_private_predict.json')
predictions.head()

,uniq_id,box
0,55555550,"[536.34716796875, 292.45703125, 33.96224594116..."
1,55555560,"[12.17514419555664, 222.23342895507812, 14.097..."
2,55555570,"[286.3392333984375, 465.8594665527344, 98.7229..."
3,55555580,"[239.1425323486328, 290.18621826171875, 234.13..."
4,55555590,"[333.85528564453125, 413.3141174316406, 169.17..."


In [55]:
# coco_val, train
ious = [0 for i in id_pairs]
for i in range(len(id_pairs)):
    row = test_private.iloc[id_pairs[i]]
    # print(coco_val[i])
    # break
    bb1 = {'x1': row['left'], 'x2': row['right'], 'y1': row['top'], 'y2': row['bottom']}
    pred_row = predictions.iloc[i]['box']
    left = pred_row[0] - pred_row[2] / 2.
    right = pred_row[0] + pred_row[2] / 2.
    top = pred_row[1] - pred_row[3] / 2.
    bot = pred_row[1] + pred_row[3] / 2.

    # print(pred_row)
    # break
    bb2 = {'x1': left, 'x2': right, 'y1': top, 'y2': bot}
    ious[i] = get_iou(bb1, bb2)

In [56]:
for i in range(10, 101, 10):
    print(f'Pecrentle {i}-th : {np.percentile(ious, i)}')

Pecrentle 10-th : 0.0
Pecrentle 20-th : 0.22019953579847842
Pecrentle 30-th : 0.5745931229061652
Pecrentle 40-th : 0.6832530825666869
Pecrentle 50-th : 0.7439474180365657
Pecrentle 60-th : 0.7870334259761114
Pecrentle 70-th : 0.8218424366425668
Pecrentle 80-th : 0.8536606410930789
Pecrentle 90-th : 0.888252918463124
Pecrentle 100-th : 0.9755206480133078


In [57]:
np.mean(ious)

0.6016933325870206

### with newer checkpoint

In [58]:
predictions = pd.read_json('/OFA/results/refcoco/refcoco_test_private_new_predict.json')
predictions.head()

,uniq_id,box
0,55555550,"[536.34716796875, 292.45703125, 39.72941970825..."
1,55555560,"[10.893549919128418, 222.23342895507812, 14.09..."
2,55555570,"[285.05712890625, 463.2962951660156, 98.722930..."
3,55555580,"[237.64163208007812, 290.18621826171875, 234.1..."
4,55555590,"[331.9328918457031, 413.3141174316406, 169.170..."


In [59]:
# coco_val, train
ious = [0 for i in id_pairs]
for i in range(len(id_pairs)):
    row = test_private.iloc[id_pairs[i]]
    # print(coco_val[i])
    # break
    bb1 = {'x1': row['left'], 'x2': row['right'], 'y1': row['top'], 'y2': row['bottom']}
    pred_row = predictions.iloc[i]['box']
    left = pred_row[0] - pred_row[2] / 2.
    right = pred_row[0] + pred_row[2] / 2.
    top = pred_row[1] - pred_row[3] / 2.
    bot = pred_row[1] + pred_row[3] / 2.

    # print(pred_row)
    # break
    bb2 = {'x1': left, 'x2': right, 'y1': top, 'y2': bot}
    ious[i] = get_iou(bb1, bb2)

In [60]:
for i in range(10, 101, 10):
    print(f'Pecrentle {i}-th : {np.percentile(ious, i)}')

Pecrentle 10-th : 0.0
Pecrentle 20-th : 0.2001232655090211
Pecrentle 30-th : 0.5788839556457075
Pecrentle 40-th : 0.6910024793591609
Pecrentle 50-th : 0.7515120681889844
Pecrentle 60-th : 0.7944108995673553
Pecrentle 70-th : 0.8289629713853489
Pecrentle 80-th : 0.8605767205820131
Pecrentle 90-th : 0.8927994753248969
Pecrentle 100-th : 0.9826799036391453


In [61]:
np.mean(ious)

0.6059904454852495

Sized of bounding boxes in ViT

Augment

More train (???)


### with older checkpoint

In [62]:
predictions = pd.read_json('/OFA/results/refcoco/refcoco_test_private_old_predict.json')
predictions.head()

,uniq_id,box
0,55555550,"[536.34716796875, 292.45703125, 33.32144927978..."
1,55555560,"[96.11956024169922, 223.9429168701172, 14.0975..."
2,55555570,"[283.3476257324219, 463.2962951660156, 109.407..."
3,55555580,"[241.1437225341797, 286.4321594238281, 315.688..."
4,55555590,"[329.36968994140625, 413.3141174316406, 135.84..."


In [63]:
# coco_val, train
ious = [0 for i in id_pairs]
for i in range(len(id_pairs)):
    row = test_private.iloc[id_pairs[i]]
    # print(coco_val[i])
    # break
    bb1 = {'x1': row['left'], 'x2': row['right'], 'y1': row['top'], 'y2': row['bottom']}
    pred_row = predictions.iloc[i]['box']
    left = pred_row[0] - pred_row[2] / 2.
    right = pred_row[0] + pred_row[2] / 2.
    top = pred_row[1] - pred_row[3] / 2.
    bot = pred_row[1] + pred_row[3] / 2.

    # print(pred_row)
    # break
    bb2 = {'x1': left, 'x2': right, 'y1': top, 'y2': bot}
    ious[i] = get_iou(bb1, bb2)

In [64]:
for i in range(10, 101, 10):
    print(f'Pecrentle {i}-th : {np.percentile(ious, i)}')
print(f'Mean value : {np.mean(ious)}')

Pecrentle 10-th : 0.0
Pecrentle 20-th : 0.12034301194055519
Pecrentle 30-th : 0.5037952494844375
Pecrentle 40-th : 0.612764344144506
Pecrentle 50-th : 0.6775647935081459
Pecrentle 60-th : 0.7267954668956867
Pecrentle 70-th : 0.7673190843630137
Pecrentle 80-th : 0.8043789734177903
Pecrentle 90-th : 0.8459440336664855
Pecrentle 100-th : 0.9831115231117139
Mean value : 0.552905976262182
